# FAST: Feedforward-Augmented Sentence Transformers

training on all glue tasks

# IMPORTS

In [1]:
# REQUIRED IMPORTS & SETUP

import os
import pickle
import numpy as np 
import pandas as pd 
import warnings
import itertools
import csv
import random
random.seed(0)  # standardized default seed
from tqdm import tqdm
from datetime import datetime

import torch
from datasets import Dataset, load_dataset
import torch
import torch.nn.functional as F

from utils.feed_forward import FeedForward
from utils.cls import extract_cls_embeddings
from utils.mean_pooling import mean_pooling
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Set Device ##########################################################
device_name = "cpu"  # default device is CPU
if torch.cuda.is_available():
    device_name = "cuda"  # CUDA for NVIDIA GPU
elif torch.backends.mps.is_available():
    device_name = torch.device("mps")  # Metal Performance Shaders for Apple M-series GPU
device = torch.device(device_name)
print(device_name)
#######################################################################

cuda


# LOAD MODELS
Load models from HuggingFace and send to GPU device. Below are some options for models.

## Base Models

### MPNet Base

In [ ]:
from transformers import MPNetTokenizer, MPNetModel
tokenizer = MPNetTokenizer.from_pretrained('sentence-transformers/all-mpnet-base-v2')
model = MPNetModel.from_pretrained("microsoft/mpnet-base").to(device)

### Distil RoBERTa Base

In [2]:
from transformers import RobertaTokenizer, RobertaModel
tokenizer = RobertaTokenizer.from_pretrained('distilroberta-base')
model = RobertaModel.from_pretrained('distilroberta-base').to(device)

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Sentence Transformers

### MPNet Sentence Transformer

In [2]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2').to(device)

### Distil RoBERTa Sentence Transformer

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-distilroberta-v1').to(device)

## Task config

In [3]:
from collections import namedtuple
# sentence_type: ["one", "two"]
# class_type: ["binary", "multi", "regression"]
# input_size: int (represents input size of feedforward, could also be called embedding size)
# col_names: column names of relavent sentences on hugging face
TaskConfig = namedtuple("TaskConfig", ["sentence_type", "class_type", "input_size", "col_names"])
task_configs = {
    "cola": TaskConfig("one", "binary", 768, ['sentence']),
    "sst2": TaskConfig("one", "binary", 768, ['sentence']),
    "mrpc": TaskConfig("two", "binary", 768*2, ['sentence1', 'sentence2']),
    "stsb": TaskConfig("two", "regression", 768*2, ['sentence1', 'sentence2']),
    "qqp": TaskConfig("two", "binary", 768*2, ['question1', 'question2']),
    "mnli-m": TaskConfig("two", "multi", 768*2, ['premise', 'hypothesis']),
    "mnli-mm": TaskConfig("two", "multi", 768*2, ['premise', 'hypothesis']),
    "qnli": TaskConfig("two", "binary", 768*2, ['question', 'sentence']),
    "rte": TaskConfig("two", "binary", 768*2, ['premise', 'hypothesis']),
    "wnli": TaskConfig("two", "binary", 768*2, ['sentence1', 'sentence2']),
    "rte": TaskConfig("two", "binary", 768*2, ['sentence1', 'sentence2']),
    "qqp": TaskConfig("two", "binary", 768*2, ['question1', 'question2']),
}

current_task = "sst2"
current_task_config = task_configs[current_task]

## Load Dataset

In [4]:
data = load_dataset("glue", current_task)
data

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})

# GENERATE EMBEDDINGS

## Generate Embeddings for CLS

Generate vector encodings using tokenizer.

In [ ]:
X_train = data["train"]["sentence"]
X_val = data["validation"]["sentence"]
X_test = data["test"]["sentence"]

Y_train = data["train"]["label"]
Y_val = data["validation"]["label"]
Y_test = data["test"]["label"]

X_train = tokenizer(X_train, padding=True, truncation=True, return_tensors='pt')
X_val = tokenizer(X_val, padding=True, truncation=True, return_tensors='pt')
X_test = tokenizer(X_test, padding=True, truncation=True, return_tensors='pt')

Run vector encodings through model and extract CLS token from output.

In [ ]:
# EXPENSIVE OPERATION: This cell may take about 2 minutes or more to run

with torch.no_grad():
    X_train = model(**X_train)
    X_val = model(**X_val)
    X_test = model(**X_test)

X_train = extract_cls_embeddings(X_train)
X_val = extract_cls_embeddings(X_val)
X_test = extract_cls_embeddings(X_test)

Save generated CLS tokens to disk.

In [ ]:
model = "distilroberta"  # rename to model name
dataset = "cola"         # rename to dataset name

from pathlib import Path
Path("./output/cls").mkdir(parents=True, exist_ok=True)

with open(f'./output/cls/X_train_{dataset}_{model}.npy', 'wb') as X_train_file:
    np.save(X_train_file, X_train)
with open(f'./output/cls/X_val_{dataset}_{model}.npy', 'wb') as X_val_file:
    np.save(X_val_file, X_val)
with open(f'./output/cls/X_test_{dataset}_{model}.npy', 'wb') as X_test_file:
    np.save(X_test_file, X_test)
with open(f'./output/cls/Y_train_{dataset}_{model}.npy', 'wb') as Y_train_file:
    np.save(Y_train_file, Y_train)
with open(f'./output/cls/Y_val_{dataset}_{model}.npy', 'wb') as Y_val_file:
    np.save(Y_val_file, Y_val)
with open(f'./output/cls/Y_test_{dataset}_{model}.npy', 'wb') as Y_test_file:
    np.save(Y_test_file, Y_test)

Load generated CLS tokens from disk.

In [ ]:
model = "distilroberta"  # rename to model name
dataset = "cola"         # rename to dataset name

with open(f'./output/cls/X_train_{dataset}_{model}.npy', 'rb') as X_train_file:
    X_train = np.load(X_train_file)
with open(f'./output/cls/X_val_{dataset}_{model}.npy', 'rb') as X_val_file:
    X_val = np.load(X_val_file)
with open(f'./output/cls/X_test_{dataset}_{model}.npy', 'rb') as X_test_file:
    X_test = np.load(X_test_file)
with open(f'./output/cls/Y_train_{dataset}_{model}.npy', 'rb') as Y_train_file:
    Y_train = np.load(Y_train_file)
with open(f'./output/cls/Y_val_{dataset}_{model}.npy', 'rb') as Y_val_file:
    Y_val = np.load(Y_val_file)
with open(f'./output/cls/Y_test_{dataset}_{model}.npy', 'rb') as Y_test_file:
    Y_test = np.load(Y_test_file)

print(f"size of X_train: {len(X_train)}")

## Generate Mean Pooled Embeddings

In [5]:
X_train = data["train"]["sentence"]
X_val = data["validation"]["sentence"]
X_test = data["test"]["sentence"]

Y_train = data["train"]["label"]
Y_val = data["validation"]["label"]
Y_test = data["test"]["label"]

X_train = tokenizer(X_train, padding=True, truncation=True, return_tensors='pt')
X_val = tokenizer(X_val, padding=True, truncation=True, return_tensors='pt')
X_test = tokenizer(X_test, padding=True, truncation=True, return_tensors='pt')

In [ ]:
X_train_saved_mask = X_train['attention_mask'].to(device)
X_val_saved_mask = X_val['attention_mask'].to(device)
X_test_saved_mask = X_test['attention_mask'].to(device)

X_train.to(device)
X_val.to(device)
X_test.to(device)

with torch.no_grad():
    X_train = model(**X_train)
    X_val = model(**X_val)
    X_test = model(**X_test)

X_train = mean_pooling(X_train, X_train_saved_mask)
X_val = mean_pooling(X_val, X_val_saved_mask)
X_test = mean_pooling(X_test, X_test_saved_mask)

In [ ]:
model = "distilroberta"  # rename to model name
dataset = "cola"         # rename to dataset name

from pathlib import Path
Path("./output/mean_pooling").mkdir(parents=True, exist_ok=True)

with open(f'./output/mean_pooling/X_train_{dataset}_{model}.npy', 'wb') as X_train_file:
    np.save(X_train_file, X_train)
with open(f'./output/mean_pooling/X_val_{dataset}_{model}.npy', 'wb') as X_val_file:
    np.save(X_val_file, X_val)
with open(f'./output/mean_pooling/X_test_{dataset}_{model}.npy', 'wb') as X_test_file:
    np.save(X_test_file, X_test)
with open(f'./output/mean_pooling/Y_train_{dataset}_{model}.npy', 'wb') as Y_train_file:
    np.save(Y_train_file, Y_train)
with open(f'./output/mean_pooling/Y_val_{dataset}_{model}.npy', 'wb') as Y_val_file:
    np.save(Y_val_file, Y_val)
with open(f'./output/mean_pooling/Y_test_{dataset}_{model}.npy', 'wb') as Y_test_file:
    np.save(Y_test_file, Y_test)

In [ ]:
model = "distilroberta"  # rename to model name
dataset = "cola"         # rename to dataset name

with open(f'./output/mean_pooling/X_train_{dataset}_{model}.npy', 'rb') as X_train_file:
    X_train = np.load(X_train_file)
with open(f'./output/mean_pooling/X_val_{dataset}_{model}.npy', 'rb') as X_val_file:
    X_val = np.load(X_val_file)
with open(f'./output/mean_pooling/X_test_{dataset}_{model}.npy', 'rb') as X_test_file:
    X_test = np.load(X_test_file)
with open(f'./output/mean_pooling/Y_train_{dataset}_{model}.npy', 'rb') as Y_train_file:
    Y_train = np.load(Y_train_file)
with open(f'./output/mean_pooling/Y_val_{dataset}_{model}.npy', 'rb') as Y_val_file:
    Y_val = np.load(Y_val_file)
with open(f'./output/mean_pooling/Y_test_{dataset}_{model}.npy', 'rb') as Y_test_file:
    Y_test = np.load(Y_test_file)

print(f"size of X_train: {len(X_train)}")

size of X_train: 8551


## Generate Embeddings for Sentence Transformers

Generate sentence embeddings using sentence transformer model.

In [ ]:
# EXPENSIVE OPERATION: This cell may take about 2 minutes or more to run

with torch.no_grad():
    if current_task_config.sentence_type == "one":
        X_train = model.encode(data["train"][current_task_config.col_names[0]])
        X_val = model.encode(data["validation"][current_task_config.col_names[0]])
        X_test = model.encode(data["test"][current_task_config.col_names[0]])
    elif current_task_config.sentence_type == "two":
        X_train_u = model.encode(data["train"][current_task_config.col_names[0]])
        X_val_u = model.encode(data["validation"][current_task_config.col_names[0]])
        X_test_u = model.encode(data["test"][current_task_config.col_names[0]])

        X_train_v = model.encode(data["train"][current_task_config.col_names[1]])
        X_val_v = model.encode(data["validation"][current_task_config.col_names[1]])
        X_test_v = model.encode(data["test"][current_task_config.col_names[1]])

        X_train = np.concatenate([X_train_u, X_train_v], axis=1)
        X_val = np.concatenate([X_val_u, X_val_v], axis=1)
        X_test = np.concatenate([X_test_u, X_test_v], axis=1)
    else:
        raise Exception(f"{current_task_config.sentence_type}: sentence type not recognized")
    
    Y_train = np.array(data["train"]["label"])
    Y_val = np.array(data["validation"]["label"])
    Y_test = np.array(data["test"]["label"])

Save encodings and labels to disk for reuse. This is done because encoding embeddings takes a significant time but the encodings do not change throughout training, so we can cache it.

In [ ]:
model = "distilroberta"  # rename to model name

from pathlib import Path
Path("./output/fast").mkdir(parents=True, exist_ok=True)

with open(f'./output/fast/X_train_{current_task}_{model}.pt', 'wb') as X_train_file:
    torch.save(X_train, X_train_file)
with open(f'./output/fast/X_val_{current_task}_{model}.pt', 'wb') as X_val_file:
    torch.save(X_val, X_val_file)
with open(f'./output/fast/X_test_{current_task}_{model}.pt', 'wb') as X_test_file:
    torch.save(X_test, X_test_file)
with open(f'./output/fast/Y_train_{current_task}_{model}.npy', 'wb') as Y_train_file:
    np.save(Y_train_file, Y_train)
with open(f'./output/fast/Y_val_{current_task}_{model}.npy', 'wb') as Y_val_file:
    np.save(Y_val_file, Y_val)
with open(f'./output/fast/Y_test_{current_task}_{model}.npy', 'wb') as Y_test_file:
    np.save(Y_test_file, Y_test)

Load saved encodings and labels from disk, if previously saved.

In [ ]:
model = "distilroberta"  # rename to model name

with open(f'./output/fast/X_train_{current_task}_{model}.pt', 'rb') as X_train_file:
    X_train = torch.load(X_train_file)
with open(f'./output/fast/X_val_{current_task}_{model}.pt', 'rb') as X_val_file:
    X_val = torch.load(X_val_file)
with open(f'./output/fast/X_test_{current_task}_{model}.pt', 'rb') as X_test_file:
    X_test = torch.load(X_test_file)
with open(f'./output/fast/Y_train_{current_task}_{model}.npy', 'rb') as Y_train_file:
    Y_train = np.load(Y_train_file)
with open(f'./output/fast/Y_val_{current_task}_{model}.npy', 'rb') as Y_val_file:
    Y_val = np.load(Y_val_file)
with open(f'./output/fast/Y_test_{current_task}_{model}.npy', 'rb') as Y_test_file:
    Y_test = np.load(Y_test_file)

print(f"size of X_train: {len(X_train)}")

# TRAINING LOOP

## Define Hyperparameters
Defining hyperparameter grid for grid search

In [18]:
param_grid = {
    'num_epochs': [50],
    'batch_size': [32, 128, 512],
    'learning_rate': [1e-2, 1e-3],
    'category': ['C'],
    'norm': [False],
    'input_size': [current_task_config.input_size],
    'layer_size': [current_task_config.input_size],
    'num_layers': [1, 2, 3],
    'weight_decay':[1e-2, 1e-3, 1e-4],
    'patience': [3],
    'min_delta': [0],
    'device': [device_name]
}

# Create a list of all combinations of hyperparameters
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
print(f"{len(all_params)} hyperparameter combinations")

54 hyperparameter combinations


In [19]:
# for output file name
embedding_type = "meanpool"

# setup for logging
save_file_id = datetime.now().strftime("%Y%m%d_%H%M%S")
console_output_filename = f'./output/{current_task}_console_output.txt'
with open(console_output_filename, 'a') as logfile:
    logfile.write('\n\nBEGIN TRAINING LOOP\n\n')
results_filename = f'./output/val_results_{embedding_type}_{current_task}_{save_file_id}.csv'
with open(results_filename, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    headers = list(all_params[0].keys())
    writer.writerow(['mcc', 'f1', 'accuracy'] + headers)
print(f"saving results to {results_filename}")

# saves best accuracy for progress bar display
best_acc = 0
# Iterate over all combinations of hyperparameters
bar = tqdm(enumerate(all_params), total=len(all_params))
for i, params in bar:
    # formatting params to display
    print_params = params.copy()
    for param in ['category', 'device']:
        del print_params[param]
    
    # Initialize the model with current set of hyperparameters
    feed_forward = FeedForward(**params)

    # Print stats to console
    epoch, val_loss, val_accuracy, val_f1, val_mcc = feed_forward.fit(X_train, Y_train, X_val, Y_val)

    best_acc = max(best_acc, val_accuracy)
    bar.set_description(f"Best Acc: {best_acc:.5f}, Last test: {val_accuracy:.5f}")

    # Write stats to log file
    with open(console_output_filename, 'a') as logfile:
        logfile.write(f"\n\nTraining with parameters:\n{print_params}")
        logfile.write(f"\nEarly stopped on epoch: {epoch}")
        logfile.write(f"\nValidation accuracy: {val_accuracy}")
        logfile.write(f"\nValidation f1-score: {val_f1}")
        logfile.write(f"\nValidation MCC     : {val_mcc}")

    with open(results_filename, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow([val_mcc, val_f1, val_accuracy] + list(params.values()))

saving results to ./output/val_results_mp_sst2_20231127_152129.csv


  0%|          | 0/54 [00:00<?, ?it/s]


TypeError: expected np.ndarray (got BatchEncoding)

## Printing results

In [13]:
# results_df = pd.read_csv(results_filename)
results_df = pd.read_csv("output/val_results_cola_20231127_151717.csv")

metric = "mcc"
best = results_df[metric].max()
best_row = results_df[results_df[metric] == best]
print(f"Best {metric}: {best:.5f}")
print(best_row)

Best mcc: 0.52001
         mcc        f1  accuracy  num_epochs  batch_size  learning_rate  \
31  0.520013  0.863424  0.803452          50         128          0.001   

   category   norm  input_size  layer_size  num_layers  weight_decay  \
31        C  False         768         768           2         0.001   

    patience  min_delta device  
31         3          0   cuda  


# PREDICT TEST SET

In [ ]:
# manually set this based on output CSV file

best_params = {
    'num_epochs': 50,
    'batch_size': 128,
    'learning_rate': 1e-2,
    'category': 'C',
    'norm': False,
    'input_size': 768,
    'layer_size': 6,
    'num_layers': 3,
    'weight_decay':1e-2,
    'patience': 3,
    'min_delta': 0,
    'device': device_name
}

In [ ]:
best_feed_forward = FeedForward(**best_params)

X = np.concatenate((X_train, X_val), axis=0)
Y = np.concatenate((Y_train, Y_val), axis=0)

best_feed_forward.fit(X, Y)

preds = np.argmax(best_feed_forward.predict_proba(X_test), axis=1)
print(preds.shape)


df = pd.DataFrame({
    'index': range(len(preds)),
    'prediction': preds
})

random.seed()  # set random seed based on current time just to generate random file_id
random_file_id = str(round(random.random() * 10000))
random.seed(0)  # reset random seed back to standard 0 seed
# Write the DataFrame to a .tsv file, without the header and index
df.to_csv(f'CoLA_{random_file_id}.tsv', sep='\t', index=False, header=True)